## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-28-15-50-50 +0000,nypost,Mormon church in Michigan ‘actively on fire’ a...,https://nypost.com/2025/09/28/us-news/michigan...
1,2025-09-28-15-48-56 +0000,nypost,James Comey’s ex-No. 2 gives mealy-mouthed ans...,https://nypost.com/2025/09/28/us-news/comeys-e...
2,2025-09-28-15-47-37 +0000,bbc,Watch: PM refuses to rule out VAT rise ahead o...,https://www.bbc.com/news/videos/c5y86zxvmmpo?a...
3,2025-09-28-15-46-14 +0000,startribune,Live: Vikings trail the Steelers 21-6 in the N...,https://www.startribune.com/minnesota-vikings-...
4,2025-09-28-15-45-08 +0000,nypost,$1M worth of rare whiskey stolen from Washingt...,https://nypost.com/2025/09/28/us-news/1m-worth...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2314/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
76,trump,30
99,russia,9
143,dead,8
65,government,7
264,gaza,7


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
108,2025-09-28-02-26-00 +0000,wsj,President Trump has agreed to meet with the fo...,https://www.wsj.com/politics/policy/trump-to-h...,84
139,2025-09-27-21-15-59 +0000,wapo,"Trump calls for troops in Portland, escalating...",https://www.washingtonpost.com/politics/2025/0...,60
130,2025-09-27-22-19-00 +0000,wsj,"Trump Says He Is Ordering Troops to Portland, ...",https://www.wsj.com/us-news/trump-says-he-is-o...,59
124,2025-09-27-23-18-16 +0000,nyt,"Trump Says He Has Ordered Troops to Portland, ...",https://www.nytimes.com/2025/09/27/us/politics...,58
135,2025-09-27-21-42-04 +0000,bbc,Trump authorises 'full force' as Portland beco...,https://www.bbc.com/news/articles/cddmn6ge6e2o...,54


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
108,84,2025-09-28-02-26-00 +0000,wsj,President Trump has agreed to meet with the fo...,https://www.wsj.com/politics/policy/trump-to-h...
35,36,2025-09-28-13-21-43 +0000,nypost,Southport mass shooting suspect identified as ...,https://nypost.com/2025/09/28/us-news/southpor...
31,35,2025-09-28-13-48-38 +0000,nypost,Eric Adams leaves door open to dropping out of...,https://nypost.com/2025/09/28/us-news/eric-ada...
46,33,2025-09-28-12-40-10 +0000,cbc,"40 dead, 124 injured following political rally...",https://www.cbc.ca/news/world/india-political-...
94,31,2025-09-28-07-58-55 +0000,nypost,"Portland mayor condemns federal intervention, ...",https://nypost.com/2025/09/28/us-news/portland...
159,30,2025-09-27-18-06-21 +0000,bbc,Israeli strikes kill dozens in Gaza including ...,https://www.bbc.com/news/articles/c87y58jgn5lo...
74,28,2025-09-28-09-08-40 +0000,nypost,"Russia fires over 600 drones, missiles in dead...",https://nypost.com/2025/09/28/world-news/large...
103,26,2025-09-28-03-29-21 +0000,nypost,Rep. Marjorie Taylor Greene warns in X rant t...,https://nypost.com/2025/09/27/us-news/rep-marj...
1,25,2025-09-28-15-48-56 +0000,nypost,James Comey’s ex-No. 2 gives mealy-mouthed ans...,https://nypost.com/2025/09/28/us-news/comeys-e...
68,25,2025-09-28-10-00-00 +0000,latimes,A horse called Maverick was close to death. He...,https://www.latimes.com/california/story/2025-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
